In [1]:
import os 
import sys
project_path = os.path.abspath(os.path.join(os.getcwd(), '..', ''))
sys.path.append(project_path)
from python.create_dgl_dataset import TelemacDataset,create_dgl_dataset_chunked


In [3]:
!ls /work/m24046/m24046mrcr/overfit_30min/

Group_1_peak_1200.res


In [68]:
ls /work/m24046/m24046mrcr/dataset_20_min/TetQ3400rapide_chunk

In [5]:
res_list = ['/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200.res']
mesh_list = ['/work/m24046/m24046mrcr/dataset_20_min/maillage_3.slf']
cli_list = ['/work/m24046/m24046mrcr/overfit_30min/cli']
dt_list = [1]

data_folder = '/work/m24046/m24046mrcr/overfit_30min/'
dataset_name = 'Group_1_peak_1200'

create_dgl_dataset_chunked(mesh_list,res_list,cli_list,dt_list,data_folder,dataset_name, chunk_size=30)

True

In [6]:
!ls /work/m24046/m24046mrcr/overfit_30min/

Group_1_peak_1200.res	       Group_1_peak_1200_0_60-80.pkl
Group_1_peak_1200_0_0-30.pkl   Group_1_peak_1200_base.bin
Group_1_peak_1200_0_30-60.pkl  cli


In [71]:
#!mkdir /work/m24046/m24046mrcr/dataset_20_min/TetQ3400lent_chunk/

In [72]:
### On check que les files soient bonnent ie x+y = x+1 hors bd 

In [7]:
def test_function(files,q_mask):
    """
    On check que les files soient bonnent ie x+y = x+1 hors bd 
    à tolérance près à cause du casting des float 32
    """
    for file in files : 
        with open(file, 'rb') as f:
            data = pickle.load(f)
            for i in range(len(data)-1):
                h_u_v,y = data[i]
                h_u_v_2,_ = data[i+1]

                avoid_boundaries = np.logical_not(q_mask)
                test_1 = np.allclose((h_u_v+y)[avoid_boundaries,:],h_u_v_2[avoid_boundaries,:],rtol=1e-4, atol=1e-6)

                assert(test_1)

In [9]:
import dgl

In [10]:
base_graph = dgl.load_graphs('/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_base.bin')[0]

In [11]:
node_types = base_graph[0].ndata['static'][:,0:4].numpy()
q_mask = (node_types == [0, 0, 1, 0]).all(axis=1)

In [12]:
import pickle
import numpy as np
files = ['/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_0-30.pkl',
         '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_30-60.pkl',
         '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_60-80.pkl',
        
        ]
test_function(files,q_mask)

In [14]:
pickle_files = [
         '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_0-30.pkl',
         '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_30-60.pkl',
         '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_60-80.pkl',
        ]

# Load and concatenate the lists from each pickle file
combined_list = []
for file in pickle_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)
        combined_list.extend(data)
        
# Save the new data to a new pickle file
new_pickle_file = '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_0-80.pkl'
with open(new_pickle_file, 'wb') as f:
    pickle.dump(combined_list, f)

print(f"New pickle file saved as {new_pickle_file}")


New pickle file saved as /work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_0-80.pkl


In [15]:
test_function(['/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_0-80.pkl'],q_mask)

In [16]:
from python.create_dgl_dataset import TelemacDataset
from hydra.utils import to_absolute_path

In [17]:
data_dir = '/work/m24046/m24046mrcr/TetQ2500inter_1min_chunk/multimesh_2_32.bin'
dynamic_dir = '/work/m24046/m24046mrcr/overfit_30min/Group_1_peak_1200_0_0-80.pkl'

batch_size = 1
epochs = 3000
num_training_samples = 1
num_training_time_steps = 80
timestep = 1
starting_ts = 0

ckpt_path = '/work/m24046/m24046mrcr/dataset_20_min/test_create_dataset_jupyter'

In [21]:
dataset =  TelemacDataset(
            name="telemac_test",
            data_dir=to_absolute_path(data_dir),
            dynamic_data_files= [to_absolute_path(dynamic_dir)],
            split="train",
            ckpt_path=ckpt_path,
            normalize=False
        )

In [22]:
!ls /work/m24046/m24046mrcr/dataset_20_min/test_create_dataset_jupyter

edge_stats.json  node_stats.json


In [23]:
print(dataset[0])

[Graph(num_nodes=412844, num_edges=2464330,
      ndata_schemes={'x': Scheme(shape=(9,), dtype=torch.float32), 'y': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)})]


In [25]:
for i in range(len(dataset)-1):
    avoid_boundaries = np.logical_not(q_mask)
    graph = dataset[i][0]
    graph_2 = dataset[i+1][0]
    #print(dataset[i].ndata['x'][q_mask,6:9].shape)
    h_u_v = graph.ndata['x'][avoid_boundaries,6:9]
    y = graph.ndata['y'][avoid_boundaries,:]
    h_u_v_2 = graph_2.ndata['x'][avoid_boundaries,6:9]
    #print(h_u_v+y)
    #print(h_u_v_2)
    test_1 = np.allclose(h_u_v+y,h_u_v_2,rtol=1e-4, atol=1e-6)
    #print(test_1)
    assert(test_1)
    

In [26]:
import importlib
# Reload the module
importlib.reload(importlib.import_module('python.create_dgl_dataset'))

# You may need to re-import the specific classes or functions after reloading
from python.create_dgl_dataset import TelemacDataset, create_dgl_dataset_chunked

In [27]:
### On test avec le normalize et la denormalization

In [28]:
dataset_norm =  TelemacDataset(
            name="telemac_test",
            data_dir=to_absolute_path(data_dir),
            dynamic_data_files= [to_absolute_path(dynamic_dir)],
            split="train",
            ckpt_path=ckpt_path,
            normalize=True
        )

Normalizing data...


In [29]:
node_stats = dataset_norm.node_stats

h_u_v_i_0_mean = np.array([node_stats['h'].item(),node_stats['u'].item(),node_stats['v'].item()])
h_u_v_i_0_std = np.array([node_stats['h_std'].item(),node_stats['u_std'].item(),node_stats['v_std'].item()])

delta_h_u_v_i_diff_mean = np.array([node_stats['delta_h'].item(),node_stats['delta_u'].item(),node_stats['delta_v'].item()])
delta_h_u_v_i_diff_std = np.array([node_stats['delta_h_std'].item(),node_stats['delta_u_std'].item(),node_stats['delta_v_std'].item()])

def _denormalize_data(tensor,mean,std):
    assert(tensor.shape[1]==mean.shape[0])
    assert(tensor.shape[1]==std.shape[0])
    return tensor*std + mean 
    
def _normalize_data(tensor,mean,std):
    assert(tensor.shape[1]==mean.shape[0])
    assert(tensor.shape[1]==std.shape[0])
    return (tensor-mean)/std

In [32]:
avoid_boundaries = np.logical_not(q_mask)
for i in range(len(dataset)):
    graph = dataset[i][0]
    graph_norm = dataset_norm[i][0]
    test_non_norm = graph.ndata['x'][avoid_boundaries,6:9]
    test_norm = _denormalize_data(graph_norm.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)


    test_1 = np.allclose(test_non_norm,test_norm,rtol=1e-4, atol=1e-6)

    assert(test_1)
    


In [33]:
avoid_boundaries = np.logical_not(q_mask)
for i in range(len(dataset)):
    graph = dataset[i][0]
    graph_norm = dataset_norm[i][0]
    test_non_norm = graph.ndata['y'][avoid_boundaries,0:3]
    test_norm = _denormalize_data(graph_norm.ndata['y'][avoid_boundaries,0:3],delta_h_u_v_i_diff_mean,delta_h_u_v_i_diff_std)


    test_1 = np.allclose(test_non_norm,test_norm,rtol=1e-4, atol=1e-6)

    assert(test_1)

In [35]:
for i in range(len(dataset)-1):
    graph_norm = dataset_norm[i][0]
    graph_norm_2 = dataset_norm[i+1][0]
    
    avoid_boundaries = np.logical_not(q_mask)
    huv_0 = _denormalize_data(graph_norm.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)
    delta = _denormalize_data(graph_norm.ndata['y'][avoid_boundaries,0:3],delta_h_u_v_i_diff_mean,delta_h_u_v_i_diff_std)

    huv_1 = _denormalize_data(graph_norm_2.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)

    test_1 = np.allclose(huv_0+delta,huv_1,rtol=1e-4, atol=1e-6)
    assert(test_1)

In [95]:
from dgl.dataloading import GraphDataLoader

# instantiate dataloader
dataloader = GraphDataLoader(
            dataset_norm,
            batch_size=1,  
            shuffle=False,
            drop_last=False,
        )

for i in range(len(dataloader)-1):
    main_dataloader = iter(dataloader)
    for _ in range(i):
        next(main_dataloader)
    graph_1 = next(main_dataloader)
    graph_2 = next(main_dataloader)

    avoid_boundaries = np.logical_not(q_mask)
    huv_0 = _denormalize_data(graph_1.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)

    delta = _denormalize_data(graph_1.ndata['y'][avoid_boundaries,0:3],delta_h_u_v_i_diff_mean,delta_h_u_v_i_diff_std)
    huv_1 = _denormalize_data(graph_2.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)

    test_1 = np.allclose(huv_0+delta,huv_1,rtol=1e-4, atol=1e-6)
    assert(test_1)


In [96]:
from dgl.dataloading import GraphDataLoader

# instantiate dataloader
dataloader = GraphDataLoader(
            dataset_norm,
            batch_size=1,  
            shuffle=False,
            drop_last=False,
        )
main_dataloader = iter(dataloader)
for i in range(len(main_dataloader)-1):
    # Reset the dataloader to fetch only the graphs needed in the j loop
    current_graphs_loader = GraphDataLoader(
                dataset_norm,
                batch_size=1,  
                shuffle=False,
                drop_last=False,
            )
    current_dataloader_iter = iter(current_graphs_loader)
    # Advance to the i-th graph in the current_dataloader_iter
    for j in range(i+1):
        next(current_dataloader_iter)
                
    graph = next(main_dataloader)
    next_graph = next(current_dataloader_iter)
    
    
    
    avoid_boundaries = np.logical_not(q_mask)
    huv_0 = _denormalize_data(graph.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)
    delta = _denormalize_data(graph.ndata['y'][avoid_boundaries,0:3],delta_h_u_v_i_diff_mean,delta_h_u_v_i_diff_std)
    huv_1 = _denormalize_data(next_graph.ndata['x'][avoid_boundaries,6:9],h_u_v_i_0_mean,h_u_v_i_0_std)
    
    test_1 = np.allclose(huv_0+delta,huv_1,rtol=1e-4, atol=1e-6)
    assert(test_1)

In [112]:
!ls /work/m24046/m24046mrcr/dataset_20_min/main_dataset

dataset_content.txt  main_dataset.pkl


In [100]:
import pickle
pickle_files = [
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ2500inter_chunk/TetQ2500inter_0_30-105.pkl',
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ3400inter_chunk/TetQ3400inter_0_30-105.pkl',
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ1500lent_chunk/TetQ1500lent_0_30-105.pkl',
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ2500lent_chunk/TetQ2500lent_0_30-105.pkl',
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ3400lent_chunk/TetQ3400lent_0_30-105.pkl',
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ1500rapide_chunk/TetQ1500rapide_0_30-105.pkl',
         '/work/m24046/m24046mrcr/dataset_20_min/TetQ3400rapide_chunk/TetQ3400rapide_0_30-105.pkl',
        ]

# Load and concatenate the lists from each pickle file
combined_list = []
for file in pickle_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)
        combined_list.extend(data)
        
# Save the new data to a new pickle file
new_pickle_file = '/work/m24046/m24046mrcr/dataset_20_min/main_dataset/main_dataset.pkl'
with open(new_pickle_file, 'wb') as f:
    pickle.dump(combined_list, f)

print(f"New pickle file saved as {new_pickle_file}")

New pickle file saved as /work/m24046/m24046mrcr/dataset_20_min/main_dataset/main_dataset.pkl


In [101]:
!ls -lh /work/m24046/m24046mrcr/dataset_20_min/main_dataset/

total 4.9G
-rw------- 1 m24046mrcr m24046 4.9G Sep 18 17:48 main_dataset.pkl


In [109]:
# Write the list to a .txt file
with open('/work/m24046/m24046mrcr/dataset_20_min/main_dataset/dataset_content.txt', 'w') as f:
    for item in pickle_files:
        f.write("%s\n" % item)


In [113]:
!more /work/m24046/m24046mrcr/dataset_20_min/main_dataset/dataset_content.txt

/work/m24046/m24046mrcr/dataset_20_min/TetQ2500inter_chunk/TetQ2500inter_0_30-10
5.pkl
/work/m24046/m24046mrcr/dataset_20_min/TetQ3400inter_chunk/TetQ3400inter_0_30-10
5.pkl
/work/m24046/m24046mrcr/dataset_20_min/TetQ1500lent_chunk/TetQ1500lent_0_30-105.
pkl
/work/m24046/m24046mrcr/dataset_20_min/TetQ2500lent_chunk/TetQ2500lent_0_30-105.
pkl
/work/m24046/m24046mrcr/dataset_20_min/TetQ3400lent_chunk/TetQ3400lent_0_30-105.
pkl
/work/m24046/m24046mrcr/dataset_20_min/TetQ1500rapide_chunk/TetQ1500rapide_0_30-
105.pkl
/work/m24046/m24046mrcr/dataset_20_min/TetQ3400rapide_chunk/TetQ3400rapide_0_30-
105.pkl


In [119]:
!ls /work/m24046/m24046mrcr/

TetQ2500inter_1min_chunk  TetQ2500inter_1min_multimesh_base.bin  dataset_20_min
